# Запуск локальной сети

docker run --gpus=all -p 127.0.0.1:9000:8080 us-docker.pkg.dev/colab-images/public/runtime

# Разархивируем файлы с данными


Скачиваем файл

In [1]:
import os  # Импорт библиотеки для работы с операционной системой
from google.colab import files
import shutil
import zipfile
!pip install gdown
import gdown
def exist(filepath):
  if not os.path.exists(file_path):
    return True
  return False
def create_dirtree(dirname, name1, name2):
    os.makedirs(dirname)
    os.makedirs(os.path.join(dirname, name1))
    os.makedirs(os.path.join(dirname, name2))
def create_folders_if_not_exist():
    data_folder = '/content/my_data'

    train_data_folder = os.path.join(data_folder, 'Train')
    val_data_folder = os.path.join(data_folder, 'Val')
    test_data_folder = os.path.join(data_folder, 'Test')
    model_data_folder = os.path.join(data_folder, 'Models')

    if not os.path.exists(data_folder):
        os.makedirs(data_folder)

    if not os.path.exists(train_data_folder):
        create_dirtree(train_data_folder, "woman", "man")

    if not os.path.exists(val_data_folder):
        create_dirtree(val_data_folder, "woman", "man")

    if not os.path.exists(test_data_folder):
        create_dirtree(test_data_folder, "woman", "man")

    if not os.path.exists(model_data_folder):
        os.makedirs(model_data_folder)


create_folders_if_not_exist()

# Путь к папке, куда нужно переместить архив
destination_folder = '/content'

# Имя архива
archive_name = 'GenderData.zip'

# Путь к целевой папке, куда нужно разархивировать архив
extract_folder = '/content/my_data'
if not os.path.exists(archive_name):
  gdown.download('https://drive.google.com/uc?id=1EcExF7G5HOMAvZ6IZW509jivTn56uuBB')
if not os.path.exists("/my_data/GenderData"):
      # Копируем архив в другую папку
      shutil.copy(os.path.join(destination_folder, archive_name), extract_folder)
      # Разархивируем архив в целевой папке
      with zipfile.ZipFile(os.path.join(extract_folder, archive_name), 'r') as zip_ref:
          zip_ref.extractall(extract_folder)

Downloading...
From (original): https://drive.google.com/uc?id=1EcExF7G5HOMAvZ6IZW509jivTn56uuBB
From (redirected): https://drive.google.com/uc?id=1EcExF7G5HOMAvZ6IZW509jivTn56uuBB&confirm=t&uuid=0ed0b67b-e941-4610-a161-4dca31c88107
To: /content/GenderData.zip
100%|██████████| 1.40G/1.40G [00:13<00:00, 102MB/s] 


# Создаем train, val, test

In [2]:
import shutil
import os
from pathlib import Path
import random
def clear_befor_upload(dirname):
  os.system(f'rm -rf {dirname + "/man/*"}')
  os.system(f'rm -rf {dirname + "/woman/*"}')
def calculate_percentages(num1, num2, per1, per2, per3):
    val1_m = round(num1 * per1 / 100)
    val2_m = round(num1 * per2 / 100)
    val3_m = round(num1 * per3 / 100)
    # Корректируем последнее значение, чтобы сумма была равна исходному числу
    val3_m = num1 - val1_m - val2_m

    val1_w = round(num2 * per1 / 100)
    val2_w  = round(num2 * per2 / 100)
    val3_w  = round(num2 * per3 / 100)
    # Корректируем последнее значение, чтобы сумма была равна исходному числу
    val3_w  = num2 - val1_w  - val2_w

    arr_train = [val1_m, val1_w]
    arr_val = [val2_m, val2_w]
    arr_test = [val3_m, val3_w]
    return arr_train, arr_val, arr_test

def count_files_in_directory(directory):
    return len(list(Path(directory + "/woman").rglob('*.*'))) + len(list(Path(directory + "/man").rglob('*.*')))

def move_images(num_files, source_dir, target_dir):
    files_of_man = os.listdir(source_dir + "/man")
    files_of_woman = os.listdir(source_dir + "/woman")
    # Перемешиваем список файлов
    random.shuffle(files_of_man)
    random.shuffle(files_of_woman)
    num_files_man = num_files[0]
    num_files_woman = num_files[1]
    for i in range(num_files_man):
        shutil.move(os.path.join(source_dir + "/man", files_of_man[i]), os.path.join(target_dir + "/man", files_of_man[i]))
    for i in range(num_files_woman):
        shutil.move(os.path.join(source_dir + "/woman", files_of_woman[i]), os.path.join(target_dir+ "/woman", files_of_woman[i]))

data_dir = "my_data/GenderData"
# Каталог с данными для обучения
train_dir = 'my_data/Train'
# Каталог с данными для проверки
val_dir = 'my_data/Val'
# Каталог с данными для тестирования
test_dir = 'my_data/Test'

#Очистка папок для новых файлов
clear_befor_upload(train_dir)
clear_befor_upload(val_dir)
clear_befor_upload(test_dir)

# Количество элементов данных в одном классе
num_images_man = count_files_in_directory(data_dir) - len(list(Path(data_dir + "/woman").rglob('*.*')))
num_images_woman = count_files_in_directory(data_dir) - len(list(Path(data_dir + "/man").rglob('*.*')))

num_of_train_data = 60 # Значение в процентах
num_of_val_data = 10 # Значение в процентах
num_of_test_data = 30 # Значение в процентах
info_str = f"num_of_train_data = {num_of_train_data}%\nnum_of_val_data = {num_of_val_data}%\nnum_of_test_data = {num_of_test_data}%\n"

num_of_train_data, num_of_val_data, num_of_test_data = calculate_percentages(num_images_man, num_images_woman, num_of_train_data, num_of_val_data, num_of_test_data)

move_images(num_of_train_data, data_dir, train_dir)
move_images(num_of_val_data, data_dir, val_dir)
move_images(num_of_test_data, data_dir, test_dir)
print(count_files_in_directory(train_dir))
print(count_files_in_directory(val_dir))
print(count_files_in_directory(test_dir))



14464
2410
7233


# Подготовка данных для тестирования и обучения

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(rescale=1. / 255)
train_dir = "/content/my_data/Train"
val_dir = "/content/my_data/Val"
test_dir = "/content/my_data/Test"
# Размеры изображения
img_width, img_height = 224, 224
# Размерность тензора на основе изображения для входных данных в нейронную сеть
# backend Tensorflow, channels_last
input_shape = (img_width,
               img_height,
               3)
# Размер мини-выборки
batch_size = 20
info_str += f"batch_size = {batch_size}\n"
# Количество изображений для обучения
nb_train_samples = count_files_in_directory(train_dir)
# Количество изображений для проверки
nb_validation_samples = count_files_in_directory(val_dir)
# Количество изображений для тестирования
nb_test_samples = count_files_in_directory(test_dir)

train_generator = datagen.flow_from_directory(train_dir,
                                              target_size = (img_width, img_height),
                                              batch_size = batch_size,
                                              class_mode = 'binary')

val_generator = datagen.flow_from_directory(val_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=batch_size,
                                            class_mode='binary')

test_generator = datagen.flow_from_directory(test_dir,
                                             target_size=(img_width, img_height),
                                             batch_size=batch_size,
                                             class_mode='binary')

Found 14464 images belonging to 2 classes.
Found 2410 images belonging to 2 classes.
Found 7232 images belonging to 2 classes.


# Создание модели


In [14]:
def save_model_sam(s):
    with open(os.path.join("/content/my_data/Models/", 'training_logs.txt'), 'a', encoding='utf-8') as f:
        print(s, file=f)
        f.write("\n\n")

def save_data(model, history, info_str, scores):
   with open(file_path, 'w', encoding='utf-8') as file:
      file.write(f"{info_str}\n")
      # Записываем данные в файл
      file.write("Эпоха\tПотери\t\t\tТочность\t\tВалидационные потери\tВалидационная точность\n")
      for epoch in range(len(history.history['loss'])):
          file.write(f"{epoch}\t{history.history['loss'][epoch]}\t{history.history['accuracy'][epoch]}\t{history.history['val_loss'][epoch]}\t{history.history['val_accuracy'][epoch]}\n")
      file.write("\n")
      model.summary(print_fn=save_model_sam)
      file.write(f"Колличество эпох = {epochs}\n")
      file.write(f"Доля верных ответов на тестовых данных, в процентах: {round(scores[1] * 100, 4)}\n")

In [ ]:
import os # Импорт библиотеки для работы с операционной системой
from keras.models import Sequential # Импорт класса для создания модели нейронной сети
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization # Импорт слоев для построения модели
from keras.optimizers import Adam # Импорт оптимизатора
from keras.losses import BinaryCrossentropy # Импорт функции потерь, для задач бинарной классификации
from keras.metrics import Accuracy # Импорт метрики, которая вычисляет точность классификации
from keras.models import load_model # Импорт функции, которая позволяет загрузить сохраненную модель Keras
import tensorflow as tf

#@markdown **Обучить модель?**
train_model = True #@param {type:"boolean"}
#@markdown **Кол-во эпох**
epochs = 20 #@param {type:"number"}
if(train_model == True):

  # Создаем модель
  model = Sequential() # Здесь мы создаем экземпляр класса Sequential, который будет нашей моделью
  # Добавление сверточного слоя
  model.add(Conv2D(32,
                   (3, 3),
                   activation = 'relu',
                   input_shape = input_shape))
  # Стабилизация работы
  model.add(BatchNormalization())
  # Добавление слоя пулинга
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Добавление еще одного сверточного слоя
  model.add(Conv2D(32,
                   (5, 5),
                   activation = 'relu',
                   input_shape = input_shape))
  model.add(BatchNormalization())
  # Добавление еще одного слоя пулинга
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Добавление еще одного сверточного слоя
  model.add(Conv2D(64,
                   (3, 3),
                   activation = 'relu',
                   input_shape = input_shape))
  model.add(BatchNormalization())
  # Добавление еще одного слоя пулинга
  model.add(MaxPooling2D(pool_size=(2, 2)))
    # Добавление еще одного сверточного слоя
  model.add(Conv2D(64,
                   (5, 5),
                   activation = 'relu',
                   input_shape = input_shape))
  model.add(BatchNormalization())
  # Добавление еще одного слоя пулинга
  model.add(MaxPooling2D(pool_size=(2, 2)))
  # Преобразование двумерных данных в одномерные
  model.add(Flatten())
  # Добавление полносвязного слоя
  model.add(Dense(200,
                  activation = 'relu'))
  model.add(Dropout(0.5))
  # Добавление выходного слоя
  model.add(Dense(1,
                  activation = 'sigmoid'))


  model.compile(loss='binary_crossentropy',optimizer="RMSprop",metrics=['accuracy'])

  history = model.fit(train_generator,
                                steps_per_epoch = nb_train_samples // batch_size,
                                epochs=epochs,
                                validation_data = val_generator,
                                validation_steps = nb_validation_samples // batch_size)

  model.save(os.path.join("/content/my_data/Models/", 'my_model.keras'))

  file_path = os.path.join("/content/my_data/Models/", 'training_logs.txt')


  # Открываем файл для записи
  with open(file_path, 'w', encoding='utf-8') as file:
      file.write(f"{info_str}")
      # Записываем данные в файл
      file.write("Эпоха\tПотери\t\t\tТочность\t\tВалидационные потери\tВалидационная точность\n")
      for epoch in range(len(history.history['loss'])):
          file.write(f"{epoch}\t{history.history['loss'][epoch]}\t{history.history['accuracy'][epoch]}\t{history.history['val_loss'][epoch]}\t{history.history['val_accuracy'][epoch]}\n")
else:
  print("Введите имя файла: ")
  str = input()
  model = load_model(str)


Тестирование модели и запись

In [ ]:
def myprint(s):
    with open(os.path.join("/content/my_data/Models/", 'training_logs.txt'), 'a', encoding='utf-8') as f:
        print(s, file=f)

from keras.models import load_model # Импорт функции, которая позволяет загрузить сохраненную модель Keras

scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)
print("Колличесво эпох = ", epochs )
print("Доля верных ответов на тестовых данных, в процентах:", round(scores[1] * 100, 4))
file_path = os.path.join("/content/my_data/Models/", 'training_logs.txt')

# Передайте эту функцию в model.summary()
model.summary(print_fn=myprint)

with open(file_path, 'a', encoding='utf-8') as file:
  file.write(f"Колличество эпох = {epochs}\n")
  file.write(f"Доля верных ответов на тестовых данных, в процентах: {round(scores[1] * 100, 4)}\n")

<ipython-input-8-cd8fef3666f1>:7: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)


Колличесво эпох =  20
Доля верных ответов на тестовых данных, в процентах: 79.3629


# Оптимизация гиперпараметров

In [15]:
import os # Импорт библиотеки для работы с операционной системой
from keras.models import Sequential # Импорт класса для создания модели нейронной сети
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization # Импорт слоев для построения модели
from keras.optimizers import Adam, Adagrad # Импорт оптимизатора
from keras.losses import BinaryCrossentropy # Импорт функции потерь, для задач бинарной классификации
from keras.metrics import Accuracy # Импорт метрики, которая вычисляет точность классификации
from keras.models import load_model # Импорт функции, которая позволяет загрузить сохраненную модель Keras
import tensorflow as tf
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])

   ############################################################################################################################
    model.add(Conv2D(filters=hp.Choice('conv1_filter', min_value=32, max_value = 128, steps = 12 ),
                     kernel_size=hp.Choice('conv1_kernel_size', values=[1, 5]),
                     activation=activation_choice,
                     input_shape=input_shape))
    # Стабилизация работы
    model.add(BatchNormalization())
    # Добавление слоя пулинга
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Добавление еще одного сверточного слоя
    model.add(Conv2D(filters=hp.Choice('conv2_filter', min_value=32, max_value = 128, steps = 12 ),
                     kernel_size=hp.Choice('conv2_kernel_size', values=[1, 5]),
                     activation=activation_choice,
                     input_shape=input_shape))
    model.add(BatchNormalization())
    # Добавление еще одного слоя пулинга
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Добавление еще одного сверточного слоя
    model.add(Conv2D(filters=hp.Choice('conv3_filter', min_value=32, max_value = 128, steps = 12 ),
                     kernel_size=hp.Choice('conv3_kernel_size', values=[1, 5]),
                     activation=activation_choice,
                     input_shape=input_shape))
    model.add(BatchNormalization())
    # Добавление еще одного слоя пулинга
    model.add(MaxPooling2D(pool_size=(2, 2)))
      # Добавление еще одного сверточного слоя
    model.add(Conv2D(filters=hp.Choice('conv4_filter', min_value=32, max_value = 128, steps = 12 ),
                     kernel_size=hp.Choice('conv4_kernel_size', values=[1, 5]),
                     activation=activation_choice,
                     input_shape=input_shape))
    model.add(BatchNormalization())
    # Добавление еще одного слоя пулинга
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Преобразование двумерных данных в одномерные
    model.add(Flatten())
    # Добавление полносвязного слоя
    model.add(Dense(units=hp.Int('dense_1_units', min_value=32, max_value=400, step=30),
                    activation=activation_choice))
    model.add(Dropout(0.5))
    # Добавление выходного слоя
    model.add(Dense(1,
                    activation = activation_choice))


    model.compile(loss='binary_crossentropy',optimizer=hp.Choice('optimizer', values=['adam','rmsprop','Adagrad']),metrics=['accuracy'])

    return model


In [ ]:
!pip install keras-tuner

from kerastuner.tuners import RandomSearch
from tensorflow.keras import layers
from tensorflow import keras

tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать -
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=40,               # максимальное количество запусков обучения
    directory='/content/my_data/Models'   # каталог, куда сохраняются обученные сети
    )
tuner.search_space_summary()
tuner.search(train_generator, steps_per_epoch = nb_train_samples // batch_size,
                                epochs=20,
                                validation_data = val_generator,
                                validation_steps = nb_validation_samples // batch_size)





In [ ]:
tuner.results_summary()

In [ ]:
model = tuner.get_best_models()

In [ ]:
model[0].save(os.path.join("/content/my_data/Models/", 'my_gen_model.keras'))

# Сохранение данных на диск

In [ ]:
import os # Импорт библиотеки для работы с операционной системой
import zipfile # Импорт библиотеки для работы с ZIP-архивами
import shutil # функции высокого уровня для работы с файлами и коллекциями файлов

with zipfile.ZipFile('save.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, dirs, files in os.walk('/content/my_data/Models'):
        for file in files:
            zipf.write(os.path.join(root, file))

    # Загрузка архива 'save.zip' на Google Drive
    shutil.move('save.zip', '/content/drive/MyDrive/CourseWork/Models/save.zip')

In [ ]:
model.summary()